In [137]:
from IPython.display import HTML
HTML('<style>div.text_cell_render{font-size:130%;}</style>')
%load_ext version_information
%version_information pandas

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.7.1 64bit [Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 7.2.0
OS Darwin 18.2.0 x86_64 i386 64bit
pandas 0.23.4
Wed Feb 13 20:14:43 2019 PST

# Goalie Pull Bayes Optimize

 - Save raw HTML from nhl.com

## Get Training Data

In [138]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import re
import datetime
import time
from collections import OrderedDict
import glob
from tqdm import tqdm_notebook
from colorama import Fore, Style

### Download HTML

In [139]:
def random_wait(mu) -> float:
    ''' Positive stochastic var with average of mu '''
    return np.random.beta(3, 3) * mu * 2

def init_sess(sess=None):
    if sess is not None:
        sess.close()
    _sess = requests.Session()
    return _sess

def get_page(sess, url, tries=0) -> str:
    if tries > 3:
        print(f'Scrape failed at URL = {url}')
        return None

    print(f'Requesting HTML for URL = {url}')
    page = sess.get(url)
    if page.status_code != 200:
        time.sleep(10)
        sess = init_sess(sess)
        return get_page(sess, url, tries+1)

    return page.text

def download_game_range(url_template, seasons, games):
    root_data_path = '../data/html'
    if not os.path.exists(root_data_path):
        os.makedirs(root_data_path)
        print(f'Making dirs {root_data_path}')
        
    request_delay = 3

    print(f'Starting data pull at {datetime.datetime.now()}')

    sess = init_sess()
    for season in seasons:
        data_path = os.path.join(root_data_path, season)
        if not os.path.exists(data_path):
            print(f'Making dirs {data_path}')
            os.makedirs(data_path)
        
        for game_num in games: 
            time.sleep(random_wait(request_delay))

            page_text = get_page(
                sess,
                url_template.format(season, game_num)
            )
            if page_text is None:
                print(f'Season = {season}')
                print(f'Max game = {game_num - 1}')
                break

            f_name = os.path.join(data_path, f'{game_num}.html')
            print(f'Writing HTML to file {f_name}')
            with open(f_name, 'w') as f:
                f.write(page_text)

    print(f'Ending data pull at {datetime.datetime.now()}')

In [34]:
url_tempalte = 'http://www.nhl.com/scores/htmlreports/{:}/PL02{:04d}.HTM'
seasons = ['20032004']
games = list(range(1, 5000))

download_game_range(url_tempalte, seasons, games)

Making dirs ../data/html
Starting data pull at 2019-02-03 12:14:32.934214
Making dirs ../data/html/20032004
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020001.HTM
Writing HTML to file ../data/html/20032004/1.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020002.HTM
Writing HTML to file ../data/html/20032004/2.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020003.HTM
Writing HTML to file ../data/html/20032004/3.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020004.HTM
Writing HTML to file ../data/html/20032004/4.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020005.HTM
Writing HTML to file ../data/html/20032004/5.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL020006.HTM
Writing HTML to file ../data/html/20032004/6.html
Requesting HTML for URL = http://www.nhl.com/scores/htmlreports/20032004/PL0

NameError: name 'download_page' is not defined

 - For 2003/2004 we got up to http://www.nhl.com/scores/htmlreports/20032004/PL021231.HTM